In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math


In [2]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [3]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [30]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

for artist in db.minedArtists.find({"totalKm": {"$gte": 300000}}) :
 title = artist["name"]   
 if os.path.exists("./bands_dicts/"+artist["name"]+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open("./bands_dicts/"+artist["name"]+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tmpOLDgig ={}
    for tour in tours:
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(my_nodesdict[idd]["data"].get("weight"))+1) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight"))+1)**2)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(tour_indice)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    # "notes" : "distance : "+str(round(dates[gig_indice]["distanceToNextGig"],2))+" km  \n"+"source : "+my_nodes[-1]["data"]["id"]+"  target : "+node["id"]+"  \ndatesource : "+dates[gig_indice -1]["datetime"].isoformat()+"  \ndatetarget : "+dates[gig_indice]["datetime"].isoformat()+"  \ngroup:"+str(tour_indice),
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    #"group" : str(tour_indice)
                    
                }
                #print edge
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT HERE
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted"+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted"+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges
    
    with open("./bands_dicts/"+artist["name"]+".obj", 'wb') as handle:
                            pickle.dump([artist["name"]+"/BETA_0.3",my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
 for key, value in my_nodesdict.iteritems():
            #print "key",key,"/////",value
            if key in my_nodesdictGLOBAL:
                #print "TROUVE"
                #print "key",key,"/////",value
                #print "GLOBAL",my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["weight"]=math.sqrt(my_nodesdictGLOBAL[key]['data']["weight"]**2+my_nodesdict[key]['data']["weight"]**2)
                if time.strptime(my_nodesdictGLOBAL[key]['data']["start"],DATETIME_FORMAT)>time.strptime(my_nodesdict[key]['data']["start"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["start"]=my_nodesdict[key]['data']["start"]
                if time.strptime(my_nodesdictGLOBAL[key]['data']["end"],DATETIME_FORMAT)<time.strptime(my_nodesdict[key]['data']["end"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["end"]=my_nodesdict[key]['data']["end"]
                my_nodesdictGLOBAL[key]['data']["notes"]+="  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]+=int(1)
                #print "GLOBAL2",my_nodesdictGLOBAL[key]
            else:
                my_nodesdictGLOBAL[key]=value
                #print my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["notes"]="  \n List of the bands that played in this venue:  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]=int(1)
for key in my_nodesdictGLOBAL:
    my_nodesdictGLOBAL[key]['data']["notes"]+="  \n TOTAL: "+str(my_nodesdictGLOBAL[key]['data']["count"])+" Bands"
    
    
with open("./my_nodesdictGLOBAL.obj", 'wb') as handle:
                            pickle.dump(my_nodesdictGLOBAL, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'the-fillmore-miami-beach-at-the-jackie-gleason-theater': {'lat': 25.7929935, 'lng': -80.1331558}, 'music-midtown-atlanta-s-music-festival': {'lat': 33.78473, 'lng': -84.38220999999999}, 'concord-pavilion': {'lat': 37.95921, 'lng': -121.938263}, 'nycb-theatre-at-westbury': {'lat': 40.77343, 'lng': -73.55858}, 'brighton-music-hall': {'lat': 42.35328, 'lng': -71.13254999999998}, 'paradisco': {'lat': 47.09572, 'lng': -119.98265800000001}, 'discovery-festival-rotterdam': {'lat': 51.91233, 'lng': 4.473299999999995}, 'shoreline-amphitheatre': {'lat': 37.42695, 'lng': -122.08067299999999}, 'uptown-amphitheatre-at-the-nc-music-factory': {'lat': 35.2391853, 'lng': -80.84550480000001}, 'bospop': {'lat': 51.25747, 'lng': 5.6898499999999785}, 'coral-sky-amphitheater': {'lat': 26.686, 'lng': -80.18671999999998}, 'palasport-olimpico': {'lat': 45.0411224, 'lng': 7.65175099999999}, 'i-love-heart-techno-france': {'lat': 43.57397, 'lng': 3.947279999999978}, 'beruna-music-festival': {'lat': 43.62932, 'l

In [ ]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [31]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('pete-s-place32-117', {'data': {'count': 1, 'end': '2017-04-01T19:00:00', 'name': 'pete-s-place', 'weight': 1.0, 'color': '#7f523f', 'notes': '  \n List of the bands that played in this venue:  \nblack-joe-lewis  \n TOTAL: 1 Bands', 'start': '2017-03-31T19:00:00', 'lat': 32.7677778, 'lng': -117.0222222, 'id': 'pete-s-place32-117'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'end': '2018-01-15T14:00:00', 'name': 'tucson-music-hall', 'weight': 7.158396516477644, 'color': '#3f7f6c', 'notes': '  \n List of the bands that played in this venue:  \ncherish-the-ladies  \nitzhak-perlman  \nbill-maher  \ncurtis-stigers  \njonathan-richman  \nthe-lettermen  \npiotr-ilich-chaikovskii  \nthe-midtown-men  \njoseph-haydn  \ngeorge-lopez  \nsuper-diamond  \njane-s-addiction  \nlindsey-stirling  \narturo-sandoval  \nlewis-black  \ndon-giovanni  \njerry-seinfeld  \nkt-tunstall  \nvideo-games-live  \nthe-ten-tenors  \nbellamy-brothers-band  \nbrian-regan  \nmichael-feinstein  \nrain  \njackso

In [32]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [33]:
print len(my_nodesdictGLOBAL)

127691


In [34]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'Ux1xzr2c2ayB5mZNwGOLOVm8myYTZGhRnR06r5gq6k_', u'userId': u'BxRn4kp4WFL92jW8g'}}


In [15]:
###IMPORT AL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk("./bands_dicts"): 
 for file in files:
  if file.endswith('.obj') == True:  
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    #print edgestop
    while True:
        try:
            #print
            create_topogram(tit, nodestop, edgestop)
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'Daniel Avery/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Daniel Avery/BETA_0.3', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'PxcKkaq6b2bZw2jwK', u'slug': u'daniel-averybeta_03', u'createdAt': u'2018-05-15T11:03:43.436Z'}, u'statusCode': 200}
topogram ID : PxcKkaq6b2bZw2jwK
167 nodes deleted
230 edges deleted
167 nodes created.
230 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/PxcKkaq6b2bZw2jwK


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'Ruthie Foster/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Ruthie Foster/BETA_0.3', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'YN4PPX5xWZMEtnjYa', u'slug': u'ruthie-fosterbeta_03', u'createdAt': u'2018-05-15T11:03:45.042Z'}, u'statusCode': 200}
topogram ID : YN4PPX5xWZMEtnjYa
529 nodes deleted
728 edges deleted
529 nodes created.


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


...

> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Cut Copy/BETA_0.3', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'PWB8uFB58tWyLYSg7', u'slug': u'cut-copybeta_03', u'createdAt': u'2018-05-15T14:18:32.094Z'}, u'statusCode': 200}
topogram ID : PWB8uFB58tWyLYSg7
277 nodes created.


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


320 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/PWB8uFB58tWyLYSg7
Creating topogram 'DJ Three/BETA_0.3'
{u'status': u'success', 'status_code': 200, u'data': u'rjWemyYBDpinKCmz7', u'statusCode': 201}
Creating topogram 'DJ Three/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'DJ Three/BETA_0.3', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'rjWemyYBDpinKCmz7', u'slug': u'dj-threebeta_03', u'createdAt': u'2018-05-15T14:18:36.148Z'}, u'statusCode': 200}
topogram ID : rjWemyYBDpinKCmz7


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


64 nodes created.
87 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/rjWemyYBDpinKCmz7
Creating topogram 'Addison Groove/BETA_0.3'
{u'status': u'success', 'status_code': 200, u'data': u'6vjfhN2vQvavi4Rn6', u'statusCode': 201}
Creating topogram 'Addison Groove/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Addison Groove/BETA_0.3', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'6vjfhN2vQvavi4Rn6', u'slug': u'addison-groovebeta_03', u'createdAt': u'2018-05-15T14:18:38.932Z'}, u'statusCode': 200}
topogram ID : 6vjfhN2vQvavi4Rn6
71 nodes created.
76 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/6vjfhN2vQvavi4Rn6


In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.3"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded